In [ ]:
import json
import os
from research.utils.data_access_utils import S3AccessUtils

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
bucket = 'aquabyte-axiom'
kwargs = {
    'Bucket': bucket,
}



In [ ]:
def get_keys():
    while True:
        resp = s3.s3_client.list_objects_v2(**kwargs)
        contents = resp.get('Contents')
        if contents:
            for content in contents:
                key = content['Key']
                if key.endswith('metadata.json'):
                    yield key
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break



In [ ]:
key_gen = get_keys()

In [ ]:
keys = []
for key in key_gen:
    keys.append(key)
    if len(keys) % 100 == 0:
        print(len(keys))

In [ ]:
ground_truth_weights = []
f = '/data/alok/biomass_estimation/playground/ground_truth_metadata.json'
for idx, key in enumerate(keys):
    if idx % 100 == 0:
        print('Percentage complete: {}%'.format(100 * idx / len(keys)))
    s3.download_from_s3('aquabyte-axiom', key, f)
    metadata = json.load(open(f))
    data = metadata.get('data')
    if data:
        weight = data.get('weightKgs')
    ground_truth_weights.append(weight)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
x = np.array(ground_truth_weights)

In [ ]:
plt.hist(x[x > 8000])
plt.show()